# Основы программирования в Python

*Автор: Анастасия Паршина, НИУ ВШЭ*  

## Web-scraping: скрэйпинг сайта с новостями

Дана ссылка на [сайт газеты "Московский комсомолец"](https://www.mk.ru/news/). Соберите ссылки на новости первой страницы, заголовок. Пройдите по ссылкам и соберите количество просмотров, хэштеги статьи и время публикации. 

In [2]:
import requests
from bs4 import BeautifulSoup 
from time import sleep
import pandas as pd

In [3]:
url = 'https://www.mk.ru/news/'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [4]:
article = soup.find_all('li', attrs = {'class' : 'news-listing__item'})[0]
article

<li class="news-listing__item">
<a class="news-listing__item-link" href="https://www.mk.ru/politics/2022/04/22/v-ldpr-vystupili-za-status-glavnogo-protivnika-rossii-dlya-ssha.html">
<span class="news-listing__item-time">18:00</span>
<h3 class="news-listing__item-title">В ЛДПР выступили за статус «главного противника России» для США</h3>
</a>
</li>

In [5]:
url0 = article.find_all('a')[0].get('href')
url0

'https://www.mk.ru/politics/2022/04/22/v-ldpr-vystupili-za-status-glavnogo-protivnika-rossii-dlya-ssha.html'

In [6]:
title = article.find('h3').text
title

'В ЛДПР выступили за статус «главного противника России» для США'

In [13]:
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)

In [14]:
views = int(soup0.find_all('span', attrs = {'class' : 'meta__item_views'})[0].text.strip())
views

3526

Можно также, например, вытащить теги внизу статьи. Однако они есть не всегда. [Пример статьи без тегов](https://www.mk.ru/social/2021/12/09/passazhiry-bez-qrkodov-mogut-lishitsya-kompensacii-za-kuplennye-bilety.html). Поэтому проверяем условие -- а есть ли вообще эти теги на странице?

In [16]:
if len(soup0.find_all('div', attrs = {'class' : 'article__tag'})) == 0:
    tags = 'Нет тегов'
else:
    tags = [i.text for i in soup0.find_all('div', attrs = {'class' : 'article__tag'})[0].find_all('a')]

In [18]:
news = {}

for a in soup.find_all('li', attrs = {'class' : 'news-listing__item'})[:4]:
    url0 = a.find_all('a')[0].get('href')
    title = a.find('h3').text
    time = a.find('span').text
    
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    views = int(soup0.find_all('span', attrs = {'class' : 'meta__item_views'})[0].text.strip())
    if len(soup0.find_all('div', attrs = {'class' : 'article__tag'})) == 0:
        tags = 'Нет тегов'
    else:
        tags = [i.text for i in soup0.find_all('div', attrs = {'class' : 'article__tag'})[0].find_all('a')]
    sleep(2)
    
    news[title] = [time, views, tags, url0]

In [19]:
# можно также сразу указывать, что у нас данные из словаря, orient='index' вместо transpose()
df = pd.DataFrame.from_dict(news, orient = 'index', columns = ['Time', 'View', 'Tags', 'URL'])
df.head(3)

,Time,View,Tags,URL
В ЛДПР выступили за статус «главного противника России» для США,18:00,2136,"[ЛДПР, Государственная Дума РФ, ДПМ, Россия, США]",https://www.mk.ru/politics/2022/04/22/v-ldpr-v...
"Ле Пен заявила, что Франция должна поспособствовать сближению России и НАТО",17:58,3138,"[НАТО, Эммануэль Макрон, Россия, Франция, Укра...",https://www.mk.ru/politics/2022/04/22/le-pen-z...
Следствие потребовало арестовать журналиста Кара-Мурзу по делу о распространении фейков,17:56,1993,[Москва],https://www.mk.ru/politics/2022/04/22/sledstvi...


In [20]:
df.to_csv('news.csv')